In [1]:
import pandas as pd
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import re
import pathlib
from sklearn.model_selection import train_test_split
from sagemaker.tuner import IntegerParameter, HyperparameterTuner, ContinuousParameter, CategoricalParameter
from utils.utils import get_logger, load_data
from utils.load_config_file import load_config_file
from warnings import filterwarnings
filterwarnings("ignore")

logger = get_logger(name=pathlib.Path("ltv-models-training.ipynb"))

CONFIG_PATH = "config/config.ini"
config = load_config_file(CONFIG_PATH)

pd.set_option('display.max_columns', 500)

In [2]:
synth_data = pd.read_csv("s3://hc-data-science/med-adv-sales-data/synthetic_data_1M_rows_2022-07-18_20071658176314.csv", low_memory= False)
synth_data

,zcta,county,state,fips,age,gender,contract_hc,carrier_hc,type_hc,plan_hc,effective_year,effective_month,effective_date,submitted_date,duration_in_months,app_to_pol,churn,cancellation,star_rating_hc,zcta_land_area_m2,county_land_area_m2,intersection_land_area_m2,intersection_pct_county_land_area,intersection_pct_zcta_land_area,latitude,longitude,usc_land_area_m2,cdc_all_teeth_lost,cdc_annual_checkup,cdc_arthritis,cdc_binge_drinking,cdc_copd,cdc_cancer_except_skin,cdc_cervical_cancer_screening,cdc_cholesterol_screening,cdc_chronic_kidney_disease,cdc_colorectal_cancer_screening,cdc_core_preventive_services_for_older_men,cdc_core_preventive_services_for_older_women,cdc_coronary_heart_disease,cdc_current_asthma,cdc_current_smoking,cdc_dental_visit,cdc_depression,cdc_diabetes,cdc_general_health,cdc_health_insurance,cdc_high_blood_pressure,cdc_high_cholesterol,cdc_mammography,cdc_mental_health,cdc_obesity,cdc_physical_health,cdc_physical_inactivity,cdc_sleep_<7_hours,cdc_stroke,cdc_taking_bp_medication,usc_housing_units,usc_occupied_housing_units,usc_median_home_value,usc_total_population,usc_sex_ratio,usc_pct_20_24,usc_pct_25_34,usc_pct_35_44,usc_pct_45_54,usc_pct_55_59,usc_pct_60_64,usc_pct_65_74,usc_pct_65_over,usc_pct_white,usc_pct_black,usc_pct_asian,usc_pct_hispanic_latino,usc_pct_american_indian_alaska_native,usc_pct_native_hawaiian_pacific_islander,usc_median_household_income,cms_mapd_lis_mrkt_share,cms_mapd_humana_mrkt_share,cms_mapd_uhc_mrkt_share,cms_mapd_aetna_mrkt_share,cms_mapd_bcbs_mrkt_share,cms_mapd_wellcare_mrkt_share,cms_mapd_cigna_mrkt_share,cms_mapd_kaiser_mrkt_share,cms_mapd_priority_mrkt_share,cms_mapd_other_mrkt_share,cms_mapd_penetration_2021,cms_mapd_penetration_2022,cms_mapd_penetration_pct_change,nyt_pct_gop,nyt_pct_dem,cms_mapd_mrkt_leader,usc_population_density_per_km2,ms_breast cancer screening,ms_colorectal cancer screening,ms_annual flu vaccine,ms_monitoring physical activity,ms_special needs plan (snp) care management,ms_care for older adults – medication review,ms_care for older adults – pain assessment,ms_osteoporosis management in women who had a fracture,ms_diabetes care – eye exam,ms_diabetes care – kidney disease monitoring,ms_diabetes care – blood sugar controlled,ms_rheumatoid arthritis management,ms_reducing the risk of falling,ms_improving bladder control,ms_medication reconciliation post-discharge,ms_statin therapy for patients with cardiovascular disease,ms_getting needed care,ms_getting appointments and care quickly,ms_customer service,ms_rating of health care quality,ms_rating of health plan,ms_care coordination,ms_complaints about the health plan,ms_members choosing to leave the plan,ms_health plan quality improvement,ms_plan makes timely decisions about appeals,ms_reviewing appeals decisions,ms_call center – foreign language interpreter and tty availability,ms_complaints about the drug plan,ms_drug plan quality improvement,ms_rating of drug plan,ms_getting needed prescription drugs,ms_mpf price accuracy,ms_medication adherence for diabetes medications,ms_medication adherence for hypertension (ras antagonists),ms_medication adherence for cholesterol (statins),ms_mtm program completion rate for cmr,ms_statin use in persons with diabetes (supd),"ds_staying healthy- screenings, tests and vaccines",ds_managing chronic (long term) conditions,ds_member experience with health plan,ds_member complaints and changes in the health plan's performance,ds_health plan customer service,ds_drug plan customer service,ds_member complaints and changes in the drug plan’s performance,ds_member experience with the drug plan,ds_drug safety and accuracy of drug pricing,"dr_problems getting needed care, coverage, and cost information",dr_problems with coverage of doctors and hospitals,dr_financial reasons for disenrollment,dr_problems with prescription drug benefits and coverage,dr_problems getting information and help from the plan,ms_improving or maintaining physical health,ms_improving or maintaining mental

In [3]:
synth_data.submitted_date

0         20201007
1         20200522
2         20210303
3         20211021
4         20210616
            ...   
999995    20200918
999996    20191224
999997    20211031
999998    20170419
999999    20200829
Name: submitted_date, Length: 1000000, dtype: int64

In [18]:
### Load Dataset locally
data_path = config['ma_ltv_data_path']
data = pd.read_csv(data_path, low_memory=False)

## SAVE TO S3
s3_bucket = 'hc-data-science'
bucket_path = f"pre-conversion-ma-ltv/data/{data_path.stem}.csv"
train_data_path = 's3://{}/{}'.format(s3_bucket, bucket_path)
data.to_csv(train_data_path, index=False)

## Get data from s3
# s3_bucket = 'hc-data-science'
# bucket_path = f"pre-conversion-ma-ltv/data/ma_ltv_efft.csv"
# data_bucket_path = 's3://{}/{}'.format(s3_bucket, bucket_path)

# data = pd.read_csv(data_bucket_path, low_memory=False)
data.head()

,application_id,owner_email,application_name,policy_id,owner_id,owner_phone,parent_application_id,sk_referral_flag,bk_product_type,carrier,lead_id,first_name,last_name,submitted_weekday,submitted_day,submitted_month,submitted_year,area_code,age,age_range,jrn_response_audit_authentic,jrn_response_audit_consumer_five_minutes,jrn_response_audit_consumer_hour,jrn_response_audit_consumer_twelve_hours,jrn_response_audit_consumer_twelve_consumer_day,jrn_response_audit_consumer_week,jrn_response_audit_data_integrity,jrn_response_audit_device_five_minutes,jrn_response_audit_device_hour,jrn_response_audit_device_twelve_hours,jrn_response_audit_device_day,jrn_response_audit_device_week,jrn_response_audit_consumer_dupe_check,jrn_response_audit_entity_value,jrn_response_audit_ip_five_minutes,jrn_response_audit_ip_hour,jrn_response_audit_ip_twelve_hours,jrn_response_audit_ip_day,jrn_response_audit_ip_week,jrn_response_audit_lead_age,jrn_response_audit_age,jrn_response_audit_lead_duration,jrn_response_audit_duration,jrn_response_audit_lead_dupe_check,jrn_response_audit_lead_dupe,jrn_response_audit_lead_five_minutes,jrn_response_audit_lead_hour,jrn_response_audit_lead_twelve_hours,jrn_response_audit_lead_day,jrn_response_audit_lead_week,zcta_latitude,zcta_longitude,zcta_cdc_all_teeth_lost,zcta_cdc_annual_checkup,zcta_cdc_arthritis,zcta_cdc_binge_drinking,zcta_cdc_cancer_except_skin,zcta_cdc_cervical_cancer_screening,zcta_cdc_cholesterol_screening,zcta_cdc_chronic_kidney_disease,zcta_cdc_colorectal_cancer_screening,zcta_cdc_copd,zcta_cdc_core_preventive_services_for_older_men,zcta_cdc_core_preventive_services_for_older_women,zcta_cdc_coronary_heart_disease,zcta_cdc_current_asthma,zcta_cdc_current_smoking,zcta_cdc_dental_visit,zcta_cdc_depression,zcta_cdc_diabetes,zcta_cdc_general_health,zcta_cdc_health_insurance,zcta_cdc_high_blood_pressure,zcta_cdc_high_cholesterol,zcta_cdc_mammography,zcta_cdc_mental_health,zcta_cdc_obesity,zcta_cdc_physical_health,zcta_cdc_physical_inactivity,zcta_cdc_sleep_lt_7_hours,zcta_cdc_stroke,zcta_cdc_taking_bp_medication,zcta_cms_mapd_aetna_mrkt_share,zcta_cms_mapd_bcbs_mrkt_share,zcta_cms_mapd_cigna_mrkt_share,zcta_cms_mapd_humana_mrkt_share,zcta_cms_mapd_kaiser_mrkt_share,zcta_cms_mapd_lis_mrkt_share,zcta_cms_mapd_mrkt_leader,zcta_cms_mapd_other_mrkt_share,zcta_cms_mapd_penetration_2021,zcta_cms_mapd_penetration_2022,zcta_cms_mapd_penetration_pct_change,zcta_cms_mapd_priority_mrkt_share,zcta_cms_mapd_uhc_mrkt_share,zcta_cms_mapd_wellcare_mrkt_share,zcta_nyt_pct_dem,zcta_nyt_pct_gop,zcta_usc_housing_units,zcta_usc_land_area_m2,zcta_usc_median_home_value,zcta_usc_median_household_income,zcta_usc_occupied_housing_units,zcta_usc_pct_20_24,zcta_usc_pct_25_34,zcta_usc_pct_35_44,zcta_usc_pct_45_54,zcta_usc_pct_55_59,zcta_usc_pct_60_64,zcta_usc_pct_65_74,zcta_usc_pct_65_over,zcta_usc_pct_american_indian_alaska_native,zcta_usc_pct_asian,zcta_usc_pct_black,zcta_usc_pct_hispanic_latino,zcta_usc_pct_native_hawaiian_pacific_islander,zcta_usc_pct_white,zcta_usc_population_density_per_km2,zcta_usc_sex_ratio,zcta_usc_total_population,tu_contact_score,tu_credit_score,tu_demo_age_years,tu_demo_income_dollars,tu_demo_children_yes,tu_demo_children_no,tu_demo_affiliation_conservative,tu_demo_affiliation_liberal,tu_demo_education_years,tu_demo_homeowner_yes,tu_demo_homeowner_no,tu_demo_homevalue_dollars,tu_demo_resident_years,tu_demo_occupation_first,post_raw_application_id,post_raw_carrier,post_raw_duration_model_predictions,post_raw_conversion_model_predictions,post_raw_cancellation_model_predictions,post_raw_year_1_commission,post_raw_year_2_commission,LTV,zip,state,city,gender
0,844307,NaN,MAPD - Med Advantage with RX Coverage,E-1305139120721,802037,7.069759e+09,844307,0,O65,O65 - United Health Care,NaN,josephine,white,1,14,12,2021,706.0,-99,Less than 65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.900708,-84.729063,22.7,79.2,32.1,14.1,8.1,82.0,89.2,4.3,

In [19]:
data.shape

(21565, 146)

### Experiment 1: All Jornaya Features set to Null

In [3]:
# jornaya_feats = [f for f in data.columns if "jrn" in f.lower()]
# for f in jornaya_feats:
#     data[f] = pd.NA


### Experiment 2: All TransUnion Features set to Null

In [4]:
# tu_feats = [f for f in data.columns if "tu_" in f.lower()]
# for f in tu_feats:
#     data[f] = pd.NA


### Experiment 3: All TransUnion + Jornaya Features set to Null

In [5]:
# tu_feats = [f for f in data.columns if "tu_" in f.lower()]
# jornaya_feats = [f for f in data.columns if "jrn" in f.lower()]
# feats = tu_feats + jornaya_feats
# for f in feats:
#     data[f] = pd.NA

### Experiment 4: Drop samples with NULL TU CREDIT SCORE

In [6]:
# data = data[data.tu_CREDIT_SCORE.notna()]
# data.reset_index(inplace=True, drop= True)
# data

### Experiment 5: All features + Carrier and Policy Effective date Information for training

#### Just changed CONFIG.INI file, and retained carrier feature


### Post- Processing on Dataset

In [7]:
list(data.columns)


['application_id',
 'owner_email',
 'application_name',
 'policy_id',
 'owner_id',
 'owner_phone',
 'parent_application_id',
 'sk_referral_flag',
 'bk_product_type',
 'carrier',
 'lead_id',
 'first_name',
 'last_name',
 'submitted_weekday',
 'submitted_day',
 'submitted_month',
 'submitted_year',
 'area_code',
 'age',
 'age_range',
 'jrn_response_audit_authentic',
 'jrn_response_audit_consumer_five_minutes',
 'jrn_response_audit_consumer_hour',
 'jrn_response_audit_consumer_twelve_hours',
 'jrn_response_audit_consumer_twelve_consumer_day',
 'jrn_response_audit_consumer_week',
 'jrn_response_audit_data_integrity',
 'jrn_response_audit_device_five_minutes',
 'jrn_response_audit_device_hour',
 'jrn_response_audit_device_twelve_hours',
 'jrn_response_audit_device_day',
 'jrn_response_audit_device_week',
 'jrn_response_audit_consumer_dupe_check',
 'jrn_response_audit_entity_value',
 'jrn_response_audit_ip_five_minutes',
 'jrn_response_audit_ip_hour',
 'jrn_response_audit_ip_twelve_hours',
 

In [8]:
len(data.carrier.unique())

12

In [20]:
## Post-process the Dataset
from utils.post_processing_utils import process_catboost

X_train, X_test = process_catboost(data=data, config_path=CONFIG_PATH, for_training=False)


# logger.info(
#     f"Splitted input dataset shapes are: \
#         X_train= {X_train.shape}, y_train= {y_train.shape},\
#             X_test= {X_test.shape}, y_test= {y_test.shape}"
# )

In [21]:
X_train.carrier.unique()

AttributeError: 'DataFrame' object has no attribute 'carrier'

In [11]:
f"pre-conversion-ma-ltv/data/post-processed/{config['train_data_path'].stem}.csv"

'pre-conversion-ma-ltv/data/post-processed/train.csv'

In [22]:
## Also store Train and Test datasets in s3
# Train
bucket_path = f"pre-conversion-ma-ltv/data/post-processed/{config['train_data_path'].stem}.csv"
train_data_path = 's3://{}/{}'.format(s3_bucket, bucket_path)
X_train.to_csv(train_data_path, index=False)
# Test
bucket_path = f"pre-conversion-ma-ltv/data/post-processed/{config['test_data_path'].stem}.csv"
test_data_path = 's3://{}/{}'.format(s3_bucket, bucket_path)
X_test.to_csv(test_data_path, index=False)

In [23]:
experiment_name = 'rb_test1'
# s3_bucket = "s3://hc-prd-mlflow-bucket"
tracking_uri = "https://mlflow.healthcare.com/"

sess = sagemaker.Session(default_bucket=s3_bucket)
subnets = ['subnet-0b8fee7c', 'subnet-da08a7f1', 'subnet-e008a7cb']
security_group_ids = ['sg-e6d64f82']
role = 'arn:aws:iam::915124832670:role/hc-sagemaker-default-execution-role' # Local
current_user_arn = boto3.resource('iam').CurrentUser().arn # Local


In [24]:
current_user_arn

'arn:aws:iam::915124832670:user/rutvik.bhende'

In [25]:
hyperparameters = {
    'tracking_uri': tracking_uri,
    'experiment_name': experiment_name,
    'user_arn': current_user_arn,
    'target': "LTV",
    'iterations': 500,
    'learning_rate': 0.01,
    'depth': 12,
    'loss_function': "RMSEWithUncertainty",
    'posterior_sampling': True,
}

hyperparameter_ranges = {
    'iterations': IntegerParameter(500, 566),
    'depth': IntegerParameter(12, 13),
    'learning_rate': CategoricalParameter([0.01, 0.02]),
    'loss_function': CategoricalParameter(["RMSE", "MAE"]), 
}


metric_definitions = [
    {'Name': 'MAE_train', 'Regex': "MAE_train: ([0-9]*.[0-9]*)"},
    {'Name': 'MAE_test', 'Regex': "MAE_test: ([0-9]*.[0-9]*)"},
    {'Name': 'RMSE_train', 'Regex': "RMSE_train: ([0-9]*.[0-9]*)"},
    {'Name': 'RMSE_test', 'Regex': "RMSE_test: ([0-9]*.[0-9]*)"},
    {'Name': 'R2_score_train', 'Regex': "R2_score_train: ([0-9]*.[0-9]*)"},
    {'Name': 'R2_score_test', 'Regex': "R2_score_test: ([0-9]*.[0-9]*)"},
    {'Name': 'test_preds_mean', 'Regex': "test_preds_mean: ([0-9]*.[0-9]*)"},
]

objective_metric_name = 'RMSE_test'
objective_type = 'Minimize'

estimator = SKLearn(
    entry_point='train_catboost.py',
    source_dir='models_py',
    role=role,
    subnets=subnets,
    security_group_ids=security_group_ids,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    instance_count=1,
    instance_type='ml.c5.xlarge',
    framework_version='0.23-1',
    py_version='py3',
)

In [26]:
estimator.fit({'train':train_data_path, 'test': test_data_path})

2022-07-08 15:07:20 Starting - Starting the training job...
2022-07-08 15:07:44 Starting - Preparing the instances for trainingProfilerReport-1657292839: InProgress
......
2022-07-08 15:08:49 Downloading - Downloading input data...
2022-07-08 15:09:24 Training - Downloading the training image...
2022-07-08 15:09:45 Training - Training image download completed. Training in progress.2022-07-08 15:09:47,922 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-07-08 15:09:47,925 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-08 15:09:47,934 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-07-08 15:09:50,568 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/miniconda3/bin/python -m pip install -r requirements.txt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/7

In [17]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=10,
    max_parallel_jobs=4,
    objective_type=objective_type,
    base_tuning_job_name='mlflow-ma-ltv-preconv'
)

In [18]:
# tuner.fit({'train':train_data_path, 'test': test_data_path})